### Exp: 1 Standard Perceptron

In [68]:
import numpy as np
class Perceptron:
    def __init__(self, lrng_rate=0.01, epochs=100):
        self.lrng_rate = lrng_rate
        self.epochs = epochs
        self.wts = None
        self.b = None
    def act_fun(self, linr_out):
        return 1 if linr_out >= 0 else 0

    def fit(self, X, y):
        self.wts = np.zeros(X.shape[1]) 
        self.b = 0  
        for epoch in range(self.epochs):
            for i in range(X.shape[0]):
                linr_out = np.dot(X[i], self.wts) + self.b
                y_pred = self.act_fun(linr_out)
                if y_pred != y[i]:
                    update = self.lrng_rate * (y[i] - y_pred)
                    self.wts += update * X[i] 
                    self.b += update 

    def predict(self, X):
        linr_out = np.dot(X, self.wts) + self.b
        return np.array([self.act_fun(x) for x in linr_out])

### Data Load

In [69]:
import pandas as pd
file_path = 'Imputed_Csv_File.csv'
data = pd.read_csv(file_path)
X = data.drop(columns=['label']).values
y = data['label'].values


### 5 - Fold accuracy , recall , f1, precision

In [70]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
def calculate_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='binary',zero_division=0)
    recall = recall_score(y_true, y_pred, average='binary')
    f1 = f1_score(y_true, y_pred, average='binary')
    return accuracy, precision, recall, f1
def cross_validate_model(model, X, y, folds=5):
    kf = KFold(n_splits=folds, shuffle=True, random_state=42)
    
    accuracies = []
    precisions = []
    recalls = []
    f1_scores = []
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy, precision, recall, f1 = calculate_metrics(y_test, y_pred)
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)
    return (np.mean(accuracies), np.mean(precisions), np.mean(recalls), np.mean(f1_scores))

In [71]:
perceptron = Perceptron(lrng_rate=0.01, epochs=100)
accuracy_cv, precision_cv, recall_cv, f1_cv = cross_validate_model(perceptron, X, y)
print(f"Accuracy: {accuracy_cv:.2f}, Precision: {precision_cv:.2f}, Recall: {recall_cv:.2f}, F1 Score: {f1_cv:.2f}")

Accuracy: 0.67, Precision: 0.58, Recall: 0.68, F1 Score: 0.58


### Exp:2 Sigmoid and binary Cross Entropy

In [43]:
import numpy as np

class PerceptronWithLoss:
    def __init__(self, lrng_rate=0.01, epochs=100):
        self.lrng_rate = lrng_rate
        self.epochs = epochs
        self.wts = None
        self.b = None 
    def act_fun(self, linr_out):
        return 1 / (1 + np.exp(-linr_out))
    def act_drtv(self, out):
        return out * (1 - out)
    def bce(self, y_true, y_pred):
        epsilon = 1e-15 
        y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
        return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

    def fit(self, X, y):
        self.wts = np.zeros(X.shape[1]) 
        self.b = 0
        for epoch in range(self.epochs):
            epoch_loss = 0
            for i in range(X.shape[0]):
                linr_out = np.dot(X[i], self.wts) + self.b
                y_pred = self.act_fun(linr_out)
                err = y[i] - y_pred
                self.wts += self.lrng_rate * err * self.act_drtv(y_pred) * X[i]
                self.b += self.lrng_rate * err * self.act_drtv(y_pred)
                epoch_loss += self.bce(y[i], y_pred)
            epoch_loss /= X.shape[0]
            if epoch % 10 == 0:
                print(f"Epoch {epoch}, Loss: {epoch_loss:.4f}")

    def predict(self, X):
        linr_out = np.dot(X, self.wts) + self.b
        y_pred = self.act_fun(linr_out)
        return np.where(y_pred >= 0.5, 1, 0)


perceptron = PerceptronWithLoss(lrng_rate=0.01, epochs=100)
accuracy_cv, precision_cv, recall_cv, f1_cv = cross_validate_model(perceptron, X, y)
print(f"Accuracy: {accuracy_cv:.2f}, Precision: {precision_cv:.2f}, Recall: {recall_cv:.2f}, F1 Score: {f1_cv:.2f}")

Epoch 0, Loss: 0.6491
Epoch 10, Loss: 0.5566
Epoch 20, Loss: 0.5267
Epoch 30, Loss: 0.5108
Epoch 40, Loss: 0.5010
Epoch 50, Loss: 0.4944
Epoch 60, Loss: 0.4897
Epoch 70, Loss: 0.4861
Epoch 80, Loss: 0.4834
Epoch 90, Loss: 0.4812
Epoch 0, Loss: 0.6540
Epoch 10, Loss: 0.5612
Epoch 20, Loss: 0.5304
Epoch 30, Loss: 0.5142
Epoch 40, Loss: 0.5045
Epoch 50, Loss: 0.4981
Epoch 60, Loss: 0.4937
Epoch 70, Loss: 0.4905
Epoch 80, Loss: 0.4881
Epoch 90, Loss: 0.4862
Epoch 0, Loss: 0.6451
Epoch 10, Loss: 0.5502
Epoch 20, Loss: 0.5182
Epoch 30, Loss: 0.5002
Epoch 40, Loss: 0.4889
Epoch 50, Loss: 0.4811
Epoch 60, Loss: 0.4755
Epoch 70, Loss: 0.4713
Epoch 80, Loss: 0.4681
Epoch 90, Loss: 0.4655
Epoch 0, Loss: 0.6550
Epoch 10, Loss: 0.5663
Epoch 20, Loss: 0.5366
Epoch 30, Loss: 0.5206
Epoch 40, Loss: 0.5108
Epoch 50, Loss: 0.5043
Epoch 60, Loss: 0.4997
Epoch 70, Loss: 0.4963
Epoch 80, Loss: 0.4938
Epoch 90, Loss: 0.4919
Epoch 0, Loss: 0.6448
Epoch 10, Loss: 0.5523
Epoch 20, Loss: 0.5223
Epoch 30, Loss: 

#### Exp4: Different Algorithms 

In [47]:
from xgboost import XGBClassifier
model = XGBClassifier(scale_pos_weight=(len(y) - sum(y)) / sum(y)) 
accuracy_cv, precision_cv, recall_cv, f1_cv = cross_validate_model(model, X, y)
print(f"Accuracy: {accuracy_cv:.2f}, Precision: {precision_cv:.2f}, Recall: {recall_cv:.2f}, F1 Score: {f1_cv:.2f}")

Accuracy: 0.74, Precision: 0.63, Recall: 0.63, F1 Score: 0.63


In [88]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(class_weight='balanced')
accuracy_cv, precision_cv, recall_cv, f1_cv = cross_validate_model(model, X, y)
print(f"Accuracy: {accuracy_cv:.2f}, Precision: {precision_cv:.2f}, Recall: {recall_cv:.2f}, F1 Score: {f1_cv:.2f}")

Accuracy: 0.76, Precision: 0.70, Recall: 0.56, F1 Score: 0.62


In [89]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(class_weight='balanced')
accuracy_cv, precision_cv, recall_cv, f1_cv = cross_validate_model(model, X, y)
print(f"Accuracy: {accuracy_cv:.2f}, Precision: {precision_cv:.2f}, Recall: {recall_cv:.2f}, F1 Score: {f1_cv:.2f}")


Accuracy: 0.76, Precision: 0.63, Recall: 0.72, F1 Score: 0.67


In [93]:
from sklearn.svm import SVC
model = SVC(kernel='rbf', class_weight='balanced')
accuracy_cv, precision_cv, recall_cv, f1_cv = cross_validate_model(model, X, y)
print(f"Accuracy: {accuracy_cv:.2f}, Precision: {precision_cv:.2f}, Recall: {recall_cv:.2f}, F1 Score: {f1_cv:.2f}")

Accuracy: 0.74, Precision: 0.61, Recall: 0.73, F1 Score: 0.66


###  Exp3: Perceptron with Weighted Loss

In [72]:
import numpy as np

class PerceptronSigmoid:
    def __init__(self, lrng_rate=0.01, epochs=80, class_weights=None):
        self.lrng_rate = lrng_rate
        self.epochs = epochs
        self.wts = None
        self.b = None
        self.class_weights = class_weights
    def act_fun(self, linr_out):
        return 1 / (1 + np.exp(-linr_out))
    def act_drtv(self, out):
        return out * (1 - out)

    def fit(self, X, y):
        self.wts = np.zeros(X.shape[1])
        self.b = 0
        if self.class_weights is None:
            unique_classes, counts = np.unique(y, return_counts=True)
            total_samples = len(y)
            self.class_weights = {cls: total_samples / count for cls, count in zip(unique_classes, counts)}
        for epoch in range(self.epochs):
            for i in range(X.shape[0]):
                linr_out = np.dot(X[i], self.wts) + self.b
                y_pred = self.act_fun(linr_out)
                err = y[i] - y_pred
                sample_weight = self.class_weights[y[i]]
                self.wts += self.lrng_rate * sample_weight * err * self.act_drtv(y_pred) * X[i]
                self.b += self.lrng_rate * sample_weight * err * self.act_drtv(y_pred)

    def predict(self, X):
        linr_out = np.dot(X, self.wts) + self.b
        y_pred = self.act_fun(linr_out)
        return np.where(y_pred >= 0.5, 1, 0)

In [87]:
class_weights = {0: 1, 1: 1.5}
model =  PerceptronSigmoid(lrng_rate=0.01, epochs=100, class_weights=class_weights) 
accuracy_cv, precision_cv, recall_cv, f1_cv = cross_validate_model(model, X, y)
print(f"Accuracy: {accuracy_cv:.2f}, Precision: {precision_cv:.2f}, Recall: {recall_cv:.2f}, F1 Score: {f1_cv:.2f}")

Accuracy: 0.76, Precision: 0.66, Recall: 0.66, F1 Score: 0.66
